***GENERATED CODE FOR furtuneglobal500classif PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import pandas as pd
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        hdfs = HDFileSystem(host=hdfs_server, port=hdfs_port)
        with hdfs.open(eval(config)['url']) as f:
            dfPd = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import round
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'name', 'transformation_label': 'String Indexer'}], 'feature': 'name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '495', 'mean': '', 'stddev': '', 'min': '3M', 'max': 'Zurich Insurance Group', 'missing': '0', 'distinct': '495'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'name'}, {'feature_label': 'name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'sector', 'transformation_label': 'String Indexer'}], 'feature': 'sector', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '495', 'mean': '', 'stddev': '', 'min': 'Aerospace & Defense', 'max': 'Wholesalers', 'missing': '0', 'distinct': '20'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'sector'}, {'feature_label': 'sector', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('sector')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'industry', 'transformation_label': 'String Indexer'}], 'feature': 'industry', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '495', 'mean': '', 'stddev': '', 'min': 'Aerospace and Defense', 'max': 'Wholesalers: Health Care', 'missing': '0', 'distinct': '56'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'industry'}, {'feature_label': 'industry', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('industry')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'revenues', 'threshold': 65327.496, 'transformation_label': 'Binarizer'}], 'feature': 'revenues', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '495', 'mean': '65735.63', 'stddev': '58736.77', 'min': '25142.0', 'max': '514405.0', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'revenues'}, {'feature_label': 'revenues', 'threshold': 65327.496, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('revenues')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'assets', 'threshold': 268385.59, 'transformation_label': 'Binarizer'}], 'feature': 'assets', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '495', 'mean': '269519.48', 'stddev': '533993.26', 'min': '4163.0', 'max': '4034481.0', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'assets'}, {'feature_label': 'assets', 'threshold': 268385.59, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('assets')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'hqcity', 'transformation_label': 'String Indexer'}], 'feature': 'hqcity', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '495', 'mean': '', 'stddev': '', 'min': 'Abbott Park', 'max': 'Zurich', 'missing': '0', 'distinct': '228'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'hqcity'}, {'feature_label': 'hqcity', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('hqcity')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'hqstate', 'transformation_label': 'String Indexer'}], 'feature': 'hqstate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '495', 'mean': '', 'stddev': '', 'min': 'Arkansas', 'max': 'Wisconsin', 'missing': '0', 'distinct': '30'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'hqstate'}, {'feature_label': 'hqstate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('hqstate')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run furtuneglobal500classifHooks.ipynb
try:
	#sourcePreExecutionHook()

	internfortuneglobal = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/intern-fortune_global_500.csv', 'filename': 'intern-fortune_global_500.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.61.134', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(internfortuneglobal)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run furtuneglobal500classifHooks.ipynb
try:
	#transformationPreExecutionHook()

	fur = TransformationMain.run(internfortuneglobal,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "495", "mean": "2019.0", "stddev": "0.0", "min": "2019", "max": "2019", "missing": "0"}, "updatedLabel": "year"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "rank", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "495", "mean": "248.0", "stddev": "143.04", "min": "1", "max": "495", "missing": "0"}, "updatedLabel": "rank"}, {"transformationsData": [{"feature_label": "name", "transformation_label": "String Indexer"}], "feature": "name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "495", "mean": "", "stddev": "", "min": "3M", "max": "Zurich Insurance Group", "missing": "0", "distinct": "495"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "name"}, {"transformationsData": [{"feature_label": "sector", "transformation_label": "String Indexer"}], "feature": "sector", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "495", "mean": "", "stddev": "", "min": "Aerospace & Defense", "max": "Wholesalers", "missing": "0", "distinct": "20"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "sector"}, {"transformationsData": [{"feature_label": "industry", "transformation_label": "String Indexer"}], "feature": "industry", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "495", "mean": "", "stddev": "", "min": "Aerospace and Defense", "max": "Wholesalers: Health Care", "missing": "0", "distinct": "56"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "industry"}, {"transformationsData": [{"feature_label": "revenues", "threshold": 65327.496, "transformation_label": "Binarizer"}], "feature": "revenues", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "495", "mean": "65735.63", "stddev": "58736.77", "min": "25142.0", "max": "514405.0", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "revenues"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "revchange", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "495", "mean": "9.12", "stddev": "15.81", "min": "-70.0", "max": "144.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "revchange"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "profits", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "495", "mean": "4328.11", "stddev": "8090.77", "min": "-22355.0", "max": "110974.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "profits"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "prftchange", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "495", "mean": "71.6", "stddev": "722.9", "min": "-1063.0", "max": "12605.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "prftchange"}, {"transformationsData": [{"feature_label": "assets", "threshold": 268385.59, "transformation_label": "Binarizer"}], "feature": "assets", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "495", "mean": "269519.48", "stddev": "533993.26", "min": "4163.0", "max": "4034481.0", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "assets"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "employees", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "495", "mean": "139499.88", "stddev": "167621.26", "min": "383", "max": "2200000", "missing": "0"}, "updatedLabel": "employees"}, {"transformationsData": [{"feature_label": "hqcity", "transformation_label": "String Indexer"}], "feature": "hqcity", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "495", "mean": "", "stddev": "", "min": "Abbott Park", "max": "Zurich", "missing": "0", "distinct": "228"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "hqcity"}, {"transformationsData": [{"feature_label": "hqstate", "transformation_label": "String Indexer"}], "feature": "hqstate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "495", "mean": "", "stddev": "", "min": "Arkansas", "max": "Wisconsin", "missing": "0", "distinct": "30"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "hqstate"}]}))

	#transformationPostExecutionHook(fur)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run furtuneglobal500classifHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(fur, ["year", "rank", "revchange", "profits", "prftchange", "employees", "name_stringindexer", "sector_stringindexer", "industry_stringindexer", "assets_binarizer", "hqcity_stringindexer", "hqstate_stringindexer"], "revenues_binarizer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

